# Etapa 1 | Carregamento, Pré-Processamento Inicial e Divisão dos Dados

In [ ]:
# ETAPA 1.0 | Bibliotecas necessárias

import os
import pandas as pd
import numpy as np
import kagglehub
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

In [ ]:
# ETAPA 1.1 | Carregar dados do Kaggle

def carregarDados(caminhoData: str) -> pd.DataFrame | None:
    """
    Carrega e unifica os arquivos CSV do Kaggle dataset
    ('time_domain', 'heart_rate_non_linear', 'frequency_domain').
    """

    dataframes = {}

    try:
        csvFiles = [f for f in os.listdir(caminhoData) if f.endswith('.csv')]

        if not csvFiles:
            print("ERRO: Nenhum arquivo CSV encontrado.")
            return None

        for fileName in csvFiles:
            fullPath = os.path.join(caminhoData, fileName)
            dfTemp = pd.read_csv(fullPath)

            keyName = fileName.split('.csv')[0]
            dataframes[keyName] = dfTemp
            print(f"✅ Carregado: {keyName} - {dfTemp.shape}")

    except Exception as e:
        print(f"Erro ao carregar: {e}")
        return None

    # Tentando identificar automaticamente quais são time / nonlinear / freq
    possiveisChaves = list(dataframes.keys())

    def buscarChave(texto):
        for key in possiveisChaves:
            if texto in key.lower():
                return key
        return None

    chaveTime = buscarChave('time')
    chaveNonLinear = buscarChave('non')
    chaveFreq = buscarChave('frequency')

    if not all([chaveTime, chaveNonLinear, chaveFreq]):
        print("ERRO: Arquivos esperados não encontrados corretamente.")
        print("Arquivos encontrados:", possiveisChaves)
        return None

    dfMerged = pd.merge(
        dataframes[chaveTime],
        dataframes[chaveNonLinear],
        on='uuid',
        how='inner'
    )

    dfFinal = pd.merge(
        dfMerged,
        dataframes[chaveFreq],
        on='uuid',
        how='inner'
    )

    print(f"\n✅ Dataset unificado com sucesso: {dfFinal.shape}")

    return dfFinal


In [ ]:
# ETAPA 1.2 | Transformação inicial dos dados

def transformarDados(df: pd.DataFrame, colunaAlvo: str = 'HR'):
    """
    Separa variáveis preditoras (X) e alvo (Y),
    remove colunas irrelevantes e aplica One-Hot Encoding em 'condition'.
    """

    if df is None:
        print("ERRO: DataFrame vazio.")
        return None, None

    # Colunas que não agregam valor (apenas identificadores)
    colunasParaRemover = ['uuid', 'datasetId']

    # Variável alvo (formato coluna para álgebra linear)
    Y = df[colunaAlvo].values.reshape(-1, 1)

    # Variáveis preditoras (remove alvo + ids)
    X = df.drop(columns=[colunaAlvo] + colunasParaRemover, errors='ignore').copy()

    # One-Hot Encoding para variável categórica 'condition'
    # drop_first=True evita multicolinearidade perfeita
    if 'condition' in X.columns:
        X = pd.get_dummies(X, columns=['condition'], drop_first=True)

    return X, Y

In [ ]:
# ETAPA 1.3 | Separação em treino e teste

def dividirTreinoTeste(X, Y, tamanhoTeste=0.2, estadoAleatorio=27):
    """
    Divide os dados em conjunto de treinamento e teste.
    """

    # Separação usando proporção definida (ex: 80% treino / 20% teste)
    XTrain, XTest, YTrain, YTest = train_test_split(
        X, Y, test_size=tamanhoTeste, random_state=estadoAleatorio
    )

    print(f"\n✅ Divisão do Conjunto de Dados: CONCLUÍDA!")
    print(f"   Treino: {XTrain.shape}")
    print(f"   Teste: {XTest.shape}")

    return XTrain, XTest, YTrain, YTest

In [ ]:
# ETAPA 1.4 | Pré-processamento dos preditores
'''
def preProcessarFeatures(XTrainRaw, XTestRaw):
    """
    Aplica transformação logarítmica, imputação e padronização (scaling).
    O ajuste é feito APENAS no conjunto de treino!
    """

    # 1. Transformação Log (reduzir assimetria)
    XTrainLog = np.log1p(XTrainRaw)
    XTestLog  = np.log1p(XTestRaw)

    # 2. Imputação (usa apenas estatísticas do TREINO)
    imputer = SimpleImputer(strategy='median')
    imputer.fit(XTrainLog)

    XTrainLog = pd.DataFrame(
        imputer.transform(XTrainLog),
        columns=XTrainRaw.columns,
        index=XTrainRaw.index
    )

    XTestLog = pd.DataFrame(
        imputer.transform(XTestLog),
        columns=XTestRaw.columns,
        index=XTestRaw.index
    )

    # 3. Padronização
    scaler = StandardScaler()
    XTrainScaled = scaler.fit_transform(XTrainLog)
    XTestScaled  = scaler.transform(XTestLog)

    # Recriar DataFrames
    XTrain = pd.DataFrame(XTrainScaled, columns=XTrainLog.columns, index=XTrainLog.index)
    XTest  = pd.DataFrame(XTestScaled, columns=XTestLog.columns, index=XTestLog.index)

    return XTrain, XTest, scaler
'''



# com Yeo-Johnson!!!
def preProcessarFeatures(XTrainRaw, XTestRaw):
    """
    Aplica transformação Yeo-Johnson e padronização (scaling).
    O ajuste é feito APENAS no conjunto de treino!
    """

    # 1. Transformação Yeo-Johnson
    # Diferente do log, aceita valores negativos!!!
    # Reduz assimetria e deixa as features mais próximas de uma distribuição normal
    pt = PowerTransformer(method='yeo-johnson')

    # Ajusta APENAS no treinamento (evita vazamento de dados)
    XTrainTransf = pt.fit_transform(XTrainRaw)
    # Aplica a MESMA transformação no conjunto de teste
    XTestTransf  = pt.transform(XTestRaw)

    # Reconstrói como DataFrame (mantém colunas e índices)
    XTrainTransf = pd.DataFrame(XTrainTransf, columns=XTrainRaw.columns, index=XTrainRaw.index
    )

    XTestTransf = pd.DataFrame(XTestTransf, columns=XTestRaw.columns, index=XTestRaw.index
    )

    # 2. Padronização
    # Necessária para modelos com regularização
    scaler = StandardScaler()

    # Ajusta no treino e aplica no teste
    XTrainScaled = scaler.fit_transform(XTrainTransf)
    XTestScaled  = scaler.transform(XTestTransf)

    # Reconstrói DataFrames finais
    XTrain = pd.DataFrame(XTrainScaled, columns=XTrainTransf.columns, index=XTrainTransf.index)
    XTest  = pd.DataFrame(XTestScaled, columns=XTestTransf.columns, index=XTestTransf.index)

    return XTrain, XTest, scaler


In [ ]:
# EXECUÇÃO DA ETAPA 1 | Pipeline Completo

# 1.
path = kagglehub.dataset_download("vinayakshanawad/heart-rate-prediction-to-monitor-stress-level")
caminhoDados = os.path.join(path, 'Train Data', 'Train Data Zip')

df = carregarDados(caminhoDados)

# 2.
X, Y = transformarDados(df, colunaAlvo='HR')

# 3.
XTrainRaw, XTestRaw, YTrain, YTest = dividirTreinoTeste(
    X, Y,
    tamanhoTeste=0.2,
    estadoAleatorio=27
)

# 4.
XTrain, XTest, scaler = preProcessarFeatures(XTrainRaw, XTestRaw)

100%|██████████| 140M/140M [00:02<00:00, 65.1MB/s]

Extracting files...


✅ Carregado: time_domain_features_train - (369289, 20)
✅ Carregado: frequency_domain_features_train - (369289, 12)
✅ Carregado: heart_rate_non_linear_features_train - (369289, 7)

✅ Dataset unificado com sucesso: (369289, 37)

✅ Divisão do Conjunto de Dados: CONCLUÍDA!
   Treino: (295431, 35)
   Teste: (73858, 35)


/usr/local/lib/python3.12/dist-packages/pandas/core/internals/blocks.py:393: RuntimeWarning: invalid value encountered in log1p
  result = func(self.values, **kwargs)
